In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm
from collections import defaultdict
import numpy as np
from rdkit import Chem
from rdkit.Chem import PandasTools
import yaml
import pickle
import pandas as pd
from bigbind import *
from run import *

In [3]:
with open("../cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [12]:
df = create_struct_df(cfg)
df.sort_values(by="pocket_size_z").query("pocket != 'CARM1_HUMAN_130_485_0'")

Using cached data from create_struct_df


,lig_smiles,lig_file,lig_pdb,pocket,ex_rec_file,ex_rec_pdb,ex_rec_pocket_file,num_pocket_residues,pocket_center_x,pocket_center_y,pocket_center_z,pocket_size_x,pocket_size_y,pocket_size_z
8106,O=C(Nc1ccccc1)Nc1ccccc1,EPHB_MYCTO_1_356_0/2zjf_bsu_lig.sdf,2zjf,EPHB_MYCTO_1_356_0,EPHB_MYCTO_1_356_0/2e3j_A_rec.pdb,2e3j,EPHB_MYCTO_1_356_0/2e3j_A_rec_pocket.pdb,17,-4.1765,1.158000,19.845500,16.215000,22.296000,12.301001
12988,O=C(O)CCc1ccccc1,PAL_ANAVT_15_565_0/5ltm_hci_lig.sdf,5ltm,PAL_ANAVT_15_565_0,PAL_ANAVT_15_565_0/3czo_B_rec.pdb,3czo,PAL_ANAVT_15_565_0/3czo_B_rec_pocket.pdb,16,-26.0495,55.741501,-25.457500,17.315001,18.738998,13.021000
11445,O=[N+]([O-])c1cccc([N+](=O)[O-])c1,MANA_CELJU_38_422_0/1gw1_nin_lig.sdf,1gw1,MANA_CELJU_38_422_0,MANA_CELJU_38_422_0/1odz_A_rec.pdb,1odz,MANA_CELJU_38_422_0/1odz_A_rec_pocket.pdb,5,15.8610,33.155001,-11.984500,18.127999,16.518002,13.291000
2337,O=C(NO)c1ccccc1O,PER_COPCI_29_363_0/1ck6_sha_lig.sdf,1ck6,PER_COPCI_29_363_0,PER_COPCI_29_363_0/2e39_A_rec.pdb,2e39,PER_COPCI_29_363_0/2e39_A_rec_pocket.pdb,12,12.2385,21.881500,31.439499,17.617000,16.612999,13.806999
2338,O=C(NO)c1ccccc1,PER_COPCI_29_363_0/1hsr_bho_lig.sdf,1hsr,PER_COPCI_29_363_0,PER_COPCI_29_363_0/1ck6_A_rec.pdb,1ck6,PER_COPCI_29_363_0/1ck6_A_rec_pocket.pdb,11,12.2385,21.881500,31.439499,17.617000,16.612999,13.806999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,NCC(=O)NS(=O)(=O)OC[C@H]1O[C@@H](n2cnc3c(N)ncn...,AACL1_BRADU_2_326_0/3mf1_g5a_lig.sdf,3mf1,AACL1_BRADU_2_326_0,AACL1_BRADU_2_326_0/3mey_A_rec.pdb,3mey,AACL1_BRADU_2_326_0/3mey_A_rec_pocket.pdb,60,-10.9935,-18.000500,33.449501,36.339000,32.697001,71.721001
16842,CC(C)(COP(O)O)[C@@H](O)C(=O)NCCC(=O)NCCS,AACL1_BRADU_2_326_0/4h2w_pns_lig.sdf,4h2w,AACL1_BRADU_2_326_0,AACL1_BRADU_2_326_0/4h2s_B_rec.pdb,4h2s,AACL1_BRADU_2_326_0/4h2s_B_rec_pocket.pdb,73,-10.9935,-18.000500,33.449501,36.339000,32.697001,71.721001
16838,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)O)[C@@H]...,AACL1_BRADU_2_326_0/4h2v_amp_lig.sdf,4h2v,AACL1_BRADU_2_326_0,AACL1_BRADU_2_326_0/4h2u_A_rec.pdb,4h2u,AACL1_BRADU_2_326_0/4h2u_A_rec_pocket.pdb,75,-10.9935,-18.000500,33.449501,36.339000,32.697001,71.721001
16832,CC(C)CO[P@@](=O)(O)O[P@@](=O)(O)OC[C@H]1O[C@@H...,AACL1_BRADU_2_326_0/3pzc_coa_lig.sdf,3pzc,AACL1_BRADU_2_326_0,AACL1_BRADU_2_326_0/4h2w_B_rec.pdb,4h2w,AACL1_BRADU_2_326_0/4h2w_B_rec_pocket.pdb,74,-10.9935,-18.000500,33.449501,36.339000,32.697001,71.721001
